In [86]:
NEO4J_URI="neo4j+s://14661657.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="3eGKj898jBqIJRC8ezhv07XqXzdFmSBF667KJ9go8TQ"

In [87]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [88]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [89]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""
graph.query(moview_query)

[]

In [90]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002445A0B0090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002445AD37090>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [91]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,exclude_types=["Genre"],verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000002445ABF4650>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002445A0B0090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002445AD37090>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [92]:
chain.graph_schema

'Node properties are the following:\nMovie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT},Person {name: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)'

In [93]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [94]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

In [95]:
prompt

FewShotPromptTemplate(input_variables=['question'], examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors played in the movie Casino?', 'query': "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name"}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}], example_prompt=PromptTemplate(input_variables=['query', 'question'], template='Us

In [96]:
print(prompt.format(question="How many artists are there?", schema="foo"))

KeyError: 'name'

In [ ]:
llm
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,cypher_prompt=prompt,verbose=True)

In [ ]:
chain.invoke("Which actors played in the movie Casino?")



> Entering new GraphCypherQAChain chain...


KeyError: 'name'

In [ ]:
chain.invoke("actors who acted in multiple movies")



> Entering new GraphCypherQAChain chain...


KeyError: 'name'